In [1]:
import pandas as pd
#Load file
df = pd.read_excel('/content/Base_FINAL.xlsx')

In [2]:
# Filters
df = df[(df['AÑO'] >= 2018) & (~df['TIPO'].isin(['EST', 'OTRAS']))]

In [3]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.utils import resample
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Ensure the data is in numpy array format and correct dtype
X_features = df.drop(columns=['TIPO', 'AÑO', 'MORENA', 'PRIANPRD', 'OTROS'])
y = df[['MORENA', 'PRIANPRD', 'OTROS']]

# Convert categorical 'EDO' column to numeric using one-hot encoding without dropping first level
X_features = pd.get_dummies(X_features, columns=['EDO']).values.astype(np.float32)
y = y.values.astype(np.float32)

# Verify the data types
print(f"X_features dtype: {X_features.dtype}")
print(f"y dtype: {y.dtype}")

# Define function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred, threshold=1e-2):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true > threshold
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# Define and train the Sequential Neural Network model
def build_sequential_nn(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

# Define and train the LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, input_shape)))
    model.add(Dense(3, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

# Train and evaluate models
input_shape = (X_features.shape[1],)
X_features_reshaped = X_features.reshape((X_features.shape[0], 1, X_features.shape[1]))

models = {
    'Sequential_NN': build_sequential_nn(input_shape),
    'LSTM': build_lstm_model(X_features.shape[1])
}

trained_models = {}
mse_scores = {}
mae_scores = {}
mape_scores = {}

# Training models
for name, model in models.items():
    if name == 'LSTM':
        model.fit(X_features_reshaped, y, epochs=50, batch_size=32, verbose=0)
    else:
        model.fit(X_features, y, epochs=50, batch_size=32, verbose=0)

    preds = model.predict(X_features if name != 'LSTM' else X_features_reshaped)
    mse_scores[name] = mean_squared_error(y, preds)
    mae_scores[name] = mean_absolute_error(y, preds)
    mape_scores[name] = mean_absolute_percentage_error(y, preds)
    trained_models[name] = model

# Select the models for predictions
selected_models = ['Sequential_NN', 'LSTM']

# Create a DataFrame to display the errors of the selected models
errors_df = pd.DataFrame(
    [(model, mse_scores[model], mae_scores[model], mape_scores[model]) for model in selected_models],
    columns=['Model', 'MSE', 'MAE', 'MAPE']
)

# Display the DataFrame with the selected models and their errors
print("Selected Models with Errors:")
print(errors_df)

# Timing the process
start_time = time.time()

# Initialize dictionary to store predictions per EDO
predictions_per_edo = {}
ci_per_edo = {}

# Get unique labels in the 'EDO' column
unique_edos = df['EDO'].unique()

# Ensure the one-hot encoded columns exist
encoded_columns = [f'EDO_{edo}' for edo in unique_edos]
encoded_columns_in_df = [col for col in encoded_columns if col in df.columns]

# Number of bootstrap samples
n_bootstrap_samples = 100  # Reduced to 100 for faster computation

# General prediction
general_predictions = {}
general_ci = {}

# Calculate general predictions and confidence intervals
for name in selected_models:
    model = trained_models[name]
    preds_samples = []

    for _ in tqdm(range(n_bootstrap_samples), desc=f"Bootstrap sampling for general {name}"):
        X_resampled = resample(X_features).astype(np.float32)
        if name == 'LSTM':
            X_resampled = X_resampled.reshape((X_resampled.shape[0], 1, X_resampled.shape[1]))
        preds_resampled = model.predict(X_resampled)
        preds_samples.append(preds_resampled.mean(axis=0))

    preds_samples = np.array(preds_samples)
    preds_mean = preds_samples.mean(axis=0)
    preds_lower = np.percentile(preds_samples, 2.5, axis=0)
    preds_upper = np.percentile(preds_samples, 97.5, axis=0)

    general_predictions[name] = preds_mean
    general_ci[name] = (preds_lower, preds_upper)

# Iterate over each unique 'EDO' label
for edo in tqdm(unique_edos, desc="Processing EDOs"):
    # Create a dummy data point for prediction for the given EDO
    X_new_period_edo = X_features.mean(axis=0).reshape((1, -1)).astype(np.float32)

    # Check if the one-hot encoded column for this EDO exists
    col_name = f'EDO_{edo}'
    if col_name in encoded_columns_in_df:
        col_index = df.columns.get_loc(col_name)
        X_new_period_edo[0, col_index] = 1  # Add one-hot encoded EDO column

    # Ensure the new period data has the same columns as the training set
    X_new_period_edo_reshaped = X_new_period_edo.reshape((X_new_period_edo.shape[0], 1, X_new_period_edo.shape[1]))

    # Store predictions and confidence intervals for the selected models
    for name in selected_models:
        model = trained_models[name]

        # Bootstrap sampling to calculate confidence intervals
        preds_samples = []
        for _ in range(n_bootstrap_samples):
            preds_resampled = model.predict(X_new_period_edo if name != 'LSTM' else X_new_period_edo_reshaped)
            preds_samples.append(preds_resampled.flatten())

        preds_samples = np.array(preds_samples)
        preds_mean = preds_samples.mean(axis=0)
        preds_lower = np.percentile(preds_samples, 2.5, axis=0)
        preds_upper = np.percentile(preds_samples, 97.5, axis=0)

        predictions_per_edo.setdefault(name, []).append(preds_mean)
        ci_per_edo.setdefault(name, []).append((preds_lower, preds_upper))

# Timing end
end_time = time.time()
total_time = end_time - start_time

# Convert general predictions to DataFrames and add labels
general_predictions_df = {name: pd.DataFrame([preds], columns=['MORENA', 'PRIANPRD', 'OTROS']) for name, preds in general_predictions.items()}

# Convert EDO predictions to DataFrames and add labels
predictions_df_per_edo = {name: pd.DataFrame(preds, index=unique_edos, columns=['MORENA', 'PRIANPRD', 'OTROS']) for name, preds in predictions_per_edo.items()}

# Display the general predictions and confidence intervals
for name, df in general_predictions_df.items():
    print(f"\n{name} general predictions for the next period:")
    print(df)
    lower_ci = general_ci[name][0]
    upper_ci = general_ci[name][1]
    print(f"95% Confidence Intervals for {name}:")
    print(f"Lower - MORENA: {lower_ci[0]:.4f}, PRIANPRD: {lower_ci[1]:.4f}, OTROS: {lower_ci[2]:.4f}")
    print(f"Upper - MORENA: {upper_ci[0]:.4f}, PRIANPRD: {upper_ci[1]:.4f}, OTROS: {upper_ci[2]:.4f}")

# Display the DataFrames with EDO predictions and confidence intervals
for name, df in predictions_df_per_edo.items():
    print(f"\n{name} predictions per EDO for the next period:")
    print(df)
    print(f"95% Confidence Intervals for {name}:")
    for i, edo in enumerate(unique_edos):
        lower_ci = ci_per_edo[name][i][0]
        upper_ci = ci_per_edo[name][i][1]
        print(f"{edo}: Lower - MORENA: {lower_ci[0]:.4f}, PRIANPRD: {lower_ci[1]:.4f}, OTROS: {lower_ci[2]:.4f}")
        print(f"       Upper - MORENA: {upper_ci[0]:.4f}, PRIANPRD: {upper_ci[1]:.4f}, OTROS: {upper_ci[2]:.4f}")

print(f"Total execution time: {total_time:.2f} seconds")


X_features dtype: float32
y dtype: float32
7/7 [==============================] - 0s 2ms/step
Selected Models with Errors:
           Model       MSE       MAE       MAPE
0  Sequential_NN  0.004148  0.049625  24.513263
1           LSTM  0.004266  0.050149  24.223070


Bootstrap sampling for general Sequential_NN:   0%|          | 0/100 [00:00<?, ?it/s]

7/7 [==============================] - 0s 2ms/step


Bootstrap sampling for general Sequential_NN:   2%|▏         | 2/100 [00:00<00:07, 13.34it/s]

7/7 [==============================] - 0s 1ms/step


...

1/1 [==============================] - 0s 22ms/step


Processing EDOs:  97%|█████████▋| 32/33 [06:42<00:12, 12.89s/it]

1/1 [==============================] - 0s 20ms/step


Processing EDOs: 100%|██████████| 33/33 [06:54<00:00, 12.57s/it]


Sequential_NN general predictions for the next period:
     MORENA  PRIANPRD     OTROS
0  0.523954  0.402746  0.070216
95% Confidence Intervals for Sequential_NN:
Lower - MORENA: 0.5080, PRIANPRD: 0.3908, OTROS: 0.0670
Upper - MORENA: 0.5349, PRIANPRD: 0.4206, OTROS: 0.0732

LSTM general predictions for the next period:
     MORENA  PRIANPRD     OTROS
0  0.526276  0.405451  0.066633
95% Confidence Intervals for LSTM:
Lower - MORENA: 0.5137, PRIANPRD: 0.3903, OTROS: 0.0636
Upper - MORENA: 0.5394, PRIANPRD: 0.4190, OTROS: 0.0697

Sequential_NN predictions per EDO for the next period:
      MORENA  PRIANPRD     OTROS
1   0.367633  0.362346  0.010018
2   0.367633  0.362346  0.010018
3   0.367633  0.362346  0.010018
4   0.367633  0.362346  0.010018
5   0.367633  0.362346  0.010018
6   0.367633  0.362346  0.010018
7   0.367633  0.362346  0.010018
8   0.367633  0.362346  0.010018
9   0.367633  0.362346  0.010018
10  0.367633  0.362346  0.010018
11  0.367633  0.362346  0.010018
12  0.367633  

In [3]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.utils import resample
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Ensure the data is in numpy array format and correct dtype
X_features = df.drop(columns=['TIPO', 'AÑO', 'MORENA', 'PRIANPRD', 'OTROS'])
y = df[['MORENA', 'PRIANPRD', 'OTROS']]

# Convert categorical 'EDO' column to numeric using one-hot encoding without dropping the first level
X_features = pd.get_dummies(X_features, columns=['EDO']).values.astype(np.float32)
y = y.values.astype(np.float32)

# Verify the data types
print(f"X_features dtype: {X_features.dtype}")
print(f"y dtype: {y.dtype}")

# Define function to calculate MAPE
def mean_absolute_percentage_error_custom(y_true, y_pred, threshold=1e-2):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true > threshold
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# Define and train the Sequential Neural Network model
def build_sequential_nn(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

# Define and train the LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, input_shape)))
    model.add(Dense(3, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

# Train and evaluate models
input_shape = (X_features.shape[1],)
X_features_reshaped = X_features.reshape((X_features.shape[0], 1, X_features.shape[1]))

models = {
    'Sequential_NN': build_sequential_nn(input_shape),
    'LSTM': build_lstm_model(X_features.shape[1])
}

trained_models = {}
mse_scores = {}
mae_scores = {}
mape_scores = {}

# Training models
for name, model in models.items():
    if name == 'LSTM':
        model.fit(X_features_reshaped, y, epochs=50, batch_size=32, verbose=0)
    else:
        model.fit(X_features, y, epochs=50, batch_size=32, verbose=0)

    preds = model.predict(X_features if name != 'LSTM' else X_features_reshaped)
    mse_scores[name] = mean_squared_error(y, preds)
    mae_scores[name] = mean_absolute_error(y, preds)
    mape_scores[name] = mean_absolute_percentage_error_custom(y, preds)
    trained_models[name] = model

# Create a DataFrame to display the errors of the selected models
errors_df = pd.DataFrame(
    [(model, mse_scores[model], mae_scores[model], mape_scores[model]) for model in models.keys()],
    columns=['Model', 'MSE', 'MAE', 'MAPE']
)

# Display the DataFrame with the errors
print("Selected Models with Errors:")
print(errors_df)

# Timing the process
start_time = time.time()

# Initialize dictionary to store predictions per EDO
predictions_per_edo = {}
ci_per_edo = {}

# Get unique labels in the 'EDO' column
unique_edos = df['EDO'].unique()

# Ensure the one-hot encoded columns exist
encoded_columns = [f'EDO_{edo}' for edo in unique_edos]
encoded_columns_in_df = [col for col in encoded_columns if col in df.columns]

# Number of bootstrap samples
n_bootstrap_samples = 100  # Reduced to 100 for faster computation

# General prediction
general_predictions = {}
general_ci = {}

# Calculate general predictions and confidence intervals
for name in models.keys():
    model = trained_models[name]
    preds_samples = []

    for _ in tqdm(range(n_bootstrap_samples), desc=f"Bootstrap sampling for general {name}"):
        X_resampled = resample(X_features).astype(np.float32)
        if name == 'LSTM':
            X_resampled = X_resampled.reshape((X_resampled.shape[0], 1, X_resampled.shape[1]))
        preds_resampled = model.predict(X_resampled)
        preds_samples.append(preds_resampled.mean(axis=0))

    preds_samples = np.array(preds_samples)
    preds_mean = preds_samples.mean(axis=0)
    preds_lower = np.percentile(preds_samples, 2.5, axis=0)
    preds_upper = np.percentile(preds_samples, 97.5, axis=0)

    general_predictions[name] = preds_mean
    general_ci[name] = (preds_lower, preds_upper)

# Iterate over each unique 'EDO' label
for edo in tqdm(unique_edos, desc="Processing EDOs"):
    # Create a dummy data point for prediction for the given EDO
    X_new_period_edo = X_features.mean(axis=0).reshape((1, -1)).astype(np.float32)

    # Check if the one-hot encoded column for this EDO exists
    col_name = f'EDO_{edo}'
    if col_name in encoded_columns_in_df:
        col_index = df.columns.get_loc(col_name)
        X_new_period_edo[0, col_index] = 1  # Add one-hot encoded EDO column

    # Ensure the new period data has the same columns as the training set
    X_new_period_edo_reshaped = X_new_period_edo.reshape((X_new_period_edo.shape[0], 1, X_new_period_edo.shape[1]))

    # Store predictions and confidence intervals for the selected models
    for name in models.keys():
        model = trained_models[name]

        # Bootstrap sampling to calculate confidence intervals
        preds_samples = []
        for _ in range(n_bootstrap_samples):
            preds_resampled = model.predict(X_new_period_edo if name != 'LSTM' else X_new_period_edo_reshaped)
            preds_samples.append(preds_resampled.flatten())

        preds_samples = np.array(preds_samples)
        preds_mean = preds_samples.mean(axis=0)
        preds_lower = np.percentile(preds_samples, 2.5, axis=0)
        preds_upper = np.percentile(preds_samples, 97.5, axis=0)

        predictions_per_edo.setdefault(name, []).append(preds_mean)
        ci_per_edo.setdefault(name, []).append((preds_lower, preds_upper))

# Timing end
end_time = time.time()
total_time = end_time - start_time

# Convert general predictions to DataFrames and add labels
general_predictions_df = {name: pd.DataFrame([preds], columns=['MORENA', 'PRIANPRD', 'OTROS']) for name, preds in general_predictions.items()}

# Convert EDO predictions to DataFrames and add labels
predictions_df_per_edo = {name: pd.DataFrame(preds, index=unique_edos, columns=['MORENA', 'PRIANPRD', 'OTROS']) for name, preds in predictions_per_edo.items()}

# Display the general predictions and confidence intervals
for name, df in general_predictions_df.items():
    print(f"\n{name} general predictions for the next period:")
    print(df)
    lower_ci = general_ci[name][0]
    upper_ci = general_ci[name][1]
    print(f"95% Confidence Intervals for {name}:")
    print(f"Lower - MORENA: {lower_ci[0]:.4f}, PRIANPRD: {lower_ci[1]:.4f}, OTROS: {lower_ci[2]:.4f}")
    print(f"Upper - MORENA: {upper_ci[0]:.4f}, PRIANPRD: {upper_ci[1]:.4f}, OTROS: {upper_ci[2]:.4f}")

# Display the DataFrames with EDO predictions and confidence intervals
for name, df in predictions_df_per_edo.items():
    print(f"\n{name} predictions per EDO for the next period:")
    print(df)
    print(f"95% Confidence Intervals for {name}:")
    for i, edo in enumerate(unique_edos):
        lower_ci = ci_per_edo[name][i][0]
        upper_ci = ci_per_edo[name][i][1]
        print(f"{edo}: Lower - MORENA: {lower_ci[0]:.4f}, PRIANPRD: {lower_ci[1]:.4f}, OTROS: {lower_ci[2]:.4f}")
        print(f"       Upper - MORENA: {upper_ci[0]:.4f}, PRIANPRD: {upper_ci[1]:.4f}, OTROS: {upper_ci[2]:.4f}")

print(f"Total execution time: {total_time:.2f} seconds")


X_features dtype: float32
y dtype: float32


7/7 [==============================] - 0s 2ms/step
Selected Models with Errors:
           Model       MSE       MAE       MAPE
0  Sequential_NN  0.004155  0.049426  24.274854
1           LSTM  0.004342  0.050548  24.143195


Bootstrap sampling for general Sequential_NN:   0%|          | 0/100 [00:00<?, ?it/s]

7/7 [==============================] - 0s 2ms/step


Bootstrap sampling for general Sequential_NN:   2%|▏         | 2/100 [00:00<00:07, 13.43it/s]

...

1/1 [==============================] - 0s 22ms/step


Processing EDOs:  97%|█████████▋| 32/33 [06:41<00:12, 12.84s/it]

1/1 [==============================] - 0s 22ms/step


Processing EDOs: 100%|██████████| 33/33 [06:54<00:00, 12.55s/it]


Sequential_NN general predictions for the next period:
     MORENA  PRIANPRD     OTROS
0  0.525546  0.406202  0.067395
95% Confidence Intervals for Sequential_NN:
Lower - MORENA: 0.5124, PRIANPRD: 0.3920, OTROS: 0.0639
Upper - MORENA: 0.5386, PRIANPRD: 0.4218, OTROS: 0.0708

LSTM general predictions for the next period:
    MORENA  PRIANPRD     OTROS
0  0.52857   0.40329  0.064108
95% Confidence Intervals for LSTM:
Lower - MORENA: 0.5129, PRIANPRD: 0.3875, OTROS: 0.0611
Upper - MORENA: 0.5440, PRIANPRD: 0.4206, OTROS: 0.0671

Sequential_NN predictions per EDO for the next period:
      MORENA  PRIANPRD     OTROS
1   0.382152  0.325424  0.064671
2   0.382152  0.325424  0.064671
3   0.382152  0.325424  0.064671
4   0.382152  0.325424  0.064671
5   0.382152  0.325424  0.064671
6   0.382152  0.325424  0.064671
7   0.382152  0.325424  0.064671
8   0.382152  0.325424  0.064671
9   0.382152  0.325424  0.064671
10  0.382152  0.325424  0.064671
11  0.382152  0.325424  0.064671
12  0.382152  0.

In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Function to predict 'MORENA' or 'PRIANPRD' given a specific level of 'PART'
def predict_morena_prianprd(part_value, edo, model_name, target='MORENA'):
    X_new_period_edo = X_features.mean(axis=0).reshape((1, -1)).astype(np.float32)
    col_name_edo = f'EDO_{edo}'
    col_name_part = 'PART'

    if col_name_edo in df.columns:
        col_index_edo = df.columns.get_loc(col_name_edo)
        X_new_period_edo[0, col_index_edo] = 1  # Add one-hot encoded EDO column

    # Set the value of 'PART'
    if col_name_part in df.columns:
        col_index_part = df.columns.get_loc(col_name_part)
        X_new_period_edo[0, col_index_part] = part_value

    X_new_period_edo_reshaped = X_new_period_edo.reshape((X_new_period_edo.shape[0], 1, X_new_period_edo.shape[1]))

    model = trained_models[model_name]
    if model_name == 'LSTM':
        preds = model.predict(X_new_period_edo_reshaped)
    else:
        preds = model.predict(X_new_period_edo)

    preds = preds.flatten()

    if target == 'MORENA':
        return -preds[0]  # Negative for maximization
    elif target == 'PRIANPRD':
        return -preds[1]  # Negative for maximization
    else:
        return 0

# Function to find the optimal 'PART' level for a specific EDO
def optimize_part_for_edo(edo, model_name, target='MORENA'):
    result = minimize(predict_morena_prianprd, x0=[0.5], args=(edo, model_name, target), bounds=[(0, 1)])
    return result.x[0], -result.fun

# Run optimization for each EDO
optimal_part_per_edo = {}

for edo in unique_edos:
    opt_part_morena, max_morena = optimize_part_for_edo(edo, 'Sequential_NN', 'MORENA')
    opt_part_prianprd, max_prianprd = optimize_part_for_edo(edo, 'Sequential_NN', 'PRIANPRD')

    optimal_part_per_edo[edo] = {
        'Optimal_PART_Morena': opt_part_morena,
        'Max_Morena': max_morena,
        'Optimal_PART_Prianprd': opt_part_prianprd,
        'Max_Prianprd': max_prianprd
    }

# Convert results to DataFrame for easy visualization
optimal_part_df = pd.DataFrame(optimal_part_per_edo).T
print(optimal_part_df)


1/1 [==============================] - 0s 19ms/step
    Optimal_PART_Morena  Max_Morena  Optimal_PART_Prianprd  Max_Prianprd
1                   0.5    0.382152                    0.5      0.325424
2                   0.5    0.382152                    0.5      0.325424
3                   0.5    0.382152                    0.5      0.325424
4                   0.5    0.382152                    0.5      0.325424
5                   0.5    0.382152                    0.5      0.325424
6                   0.5    0.382152                    0.5      0.325424
7                   0.5    0.382152                    0.5      0.325424
8                   0.5    0.382152                    0.5      0.325424
9                   0.5    0.382152                    0.5      0.325424
10                  0.5    0.382152                    0.5      0.325424
11                  0.5    0.382152                    0.5      0.325424
12                  0.5    0.382152                    0.5      0.325424

##...............................................................
## **All together**


In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.utils import resample
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Ensure the data is in numpy array format and correct dtype
X_features = df.drop(columns=['TIPO', 'AÑO', 'MORENA', 'PRIANPRD', 'OTROS'])
y = df[['MORENA', 'PRIANPRD', 'OTROS']]

# Convert categorical 'EDO' column to numeric using one-hot encoding without dropping the first level
X_features = pd.get_dummies(X_features, columns=['EDO']).values.astype(np.float32)
y = y.values.astype(np.float32)

# Verify the data types
print(f"X_features dtype: {X_features.dtype}")
print(f"y dtype: {y.dtype}")

# Define function to calculate MAPE
def mean_absolute_percentage_error_custom(y_true, y_pred, threshold=1e-2):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true > threshold
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# Define and train the Sequential Neural Network model
def build_sequential_nn(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

# Define and train the LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, input_shape)))
    model.add(Dense(3, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

# Train and evaluate models
input_shape = (X_features.shape[1],)
X_features_reshaped = X_features.reshape((X_features.shape[0], 1, X_features.shape[1]))

models = {
    'Sequential_NN': build_sequential_nn(input_shape),
    'LSTM': build_lstm_model(X_features.shape[1])
}

trained_models = {}
mse_scores = {}
mae_scores = {}
mape_scores = {}

# Training models
for name, model in models.items():
    if name == 'LSTM':
        model.fit(X_features_reshaped, y, epochs=50, batch_size=32, verbose=0)
    else:
        model.fit(X_features, y, epochs=50, batch_size=32, verbose=0)

    preds = model.predict(X_features if name != 'LSTM' else X_features_reshaped)
    mse_scores[name] = mean_squared_error(y, preds)
    mae_scores[name] = mean_absolute_error(y, preds)
    mape_scores[name] = mean_absolute_percentage_error_custom(y, preds)
    trained_models[name] = model

# Create a DataFrame to display the errors of the selected models
errors_df = pd.DataFrame(
    [(model, mse_scores[model], mae_scores[model], mape_scores[model]) for model in models.keys()],
    columns=['Model', 'MSE', 'MAE', 'MAPE']
)

# Display the DataFrame with the errors
print("Selected Models with Errors:")
print(errors_df)

# Initialize dictionary to store predictions per EDO
predictions_per_edo = {}
ci_per_edo = {}

# Get unique labels in the 'EDO' column
unique_edos = df['EDO'].unique()

# Ensure the one-hot encoded columns exist
encoded_columns = [f'EDO_{edo}' for edo in unique_edos]
encoded_columns_in_df = [col for col in encoded_columns if col in df.columns]

# Define PART levels to evaluate
part_levels = [0.40, 0.50, 0.60]

# Initialize dictionary to store general predictions and confidence intervals
general_predictions = {}
general_ci = {}

# Calculate general predictions and confidence intervals for each PART level
for part_value in part_levels:
    for name in models.keys():
        preds_samples = []

        for _ in range(100):  # Reduced to 100 for faster computation
            X_resampled = resample(X_features).astype(np.float32)
            col_name_part = 'PART'
            if col_name_part in df.columns:
                col_index_part = df.columns.get_loc(col_name_part)
                X_resampled[:, col_index_part] = part_value

            if name == 'LSTM':
                X_resampled = X_resampled.reshape((X_resampled.shape[0], 1, X_resampled.shape[1]))
            preds_resampled = trained_models[name].predict(X_resampled)
            preds_samples.append(preds_resampled.mean(axis=0))

        preds_samples = np.array(preds_samples)
        preds_mean = preds_samples.mean(axis=0)
        preds_lower = np.percentile(preds_samples, 2.5, axis=0)
        preds_upper = np.percentile(preds_samples, 97.5, axis=0)

        general_predictions[(name, part_value)] = preds_mean
        general_ci[(name, part_value)] = (preds_lower, preds_upper)

# Iterate over each unique 'EDO' label and PART levels
for edo in unique_edos:
    for part_value in part_levels:
        # Create a dummy data point for prediction for the given EDO
        X_new_period_edo = X_features.mean(axis=0).reshape((1, -1)).astype(np.float32)

        # Check if the one-hot encoded column for this EDO exists
        col_name_edo = f'EDO_{edo}'
        col_name_part = 'PART'
        if col_name_edo in encoded_columns_in_df:
            col_index_edo = df.columns.get_loc(col_name_edo)
            X_new_period_edo[0, col_index_edo] = 1  # Add one-hot encoded EDO column

        # Set the value of PART
        if col_name_part in df.columns:
            col_index_part = df.columns.get_loc(col_name_part)
            X_new_period_edo[0, col_index_part] = part_value

        X_new_period_edo_reshaped = X_new_period_edo.reshape((X_new_period_edo.shape[0], 1, X_new_period_edo.shape[1]))

        # Store predictions and confidence intervals for the selected models
        for name in models.keys():
            preds_samples = []
            for _ in range(100):
                preds_resampled = trained_models[name].predict(X_new_period_edo if name != 'LSTM' else X_new_period_edo_reshaped)
                preds_samples.append(preds_resampled.flatten())

            preds_samples = np.array(preds_samples)
            preds_mean = preds_samples.mean(axis=0)
            preds_lower = np.percentile(preds_samples, 2.5, axis=0)
            preds_upper = np.percentile(preds_samples, 97.5, axis=0)

            predictions_per_edo.setdefault((edo, name, part_value), []).append(preds_mean)
            ci_per_edo.setdefault((edo, name, part_value), []).append((preds_lower, preds_upper))

# Convert general predictions to DataFrames and add labels
general_predictions_df = {key: pd.DataFrame([preds], columns=['MORENA', 'PRIANPRD', 'OTROS']) for key, preds in general_predictions.items()}

# Convert EDO predictions to DataFrames and add labels
predictions_df_per_edo = {key: pd.DataFrame(preds, columns=['MORENA', 'PRIANPRD', 'OTROS']) for key, preds in predictions_per_edo.items()}

# Display the general predictions and confidence intervals
for (model_name, part_value), df in general_predictions_df.items():
    print(f"\n{model_name} general predictions for PART = {part_value}:")
    print(df)
    lower_ci = general_ci[(model_name, part_value)][0]
    upper_ci = general_ci[(model_name, part_value)][1]
    print(f"95% Confidence Intervals for {model_name} at PART = {part_value}:")
    print(f"Lower - MORENA: {lower_ci[0]:.4f}, PRIANPRD: {lower_ci[1]:.4f}, OTROS: {lower_ci[2]:.4f}")
    print(f"Upper - MORENA: {upper_ci[0]:.4f}, PRIANPRD: {upper_ci[1]:.4f}, OTROS: {upper_ci[2]:.4f}")

# Display the DataFrames with EDO predictions and confidence intervals
for (edo, model_name, part_value), df in predictions_df_per_edo.items():
    print(f"\n{model_name} predictions per EDO = {edo} for PART = {part_value}:")
    print(df)
    # Debugging prints to check ci_per_edo content
    print(f"ci_per_edo[{(edo, model_name, part_value)}]: {ci_per_edo[(edo, model_name, part_value)]}")
    if len(ci_per_edo[(edo, model_name, part_value)]) < 2:
        print(f"Error: ci_per_edo[{(edo, model_name, part_value)}] has less than 2 elements.")
        continue
    lower_ci = ci_per_edo[(edo, model_name, part_value)][0]
    upper_ci = ci_per_edo[(edo, model_name, part_value)][1]
    print(f"95% Confidence Intervals for {model_name} at PART = {part_value}, EDO = {edo}:")
    print(f"Lower - MORENA: {lower_ci[0]:.4f}, PRIANPRD: {lower_ci[1]:.4f}, OTROS: {lower_ci[2]:.4f}")
    print(f"Upper - MORENA: {upper_ci[0]:.4f}, PRIANPRD: {upper_ci[1]:.4f}, OTROS: {upper_ci[2]:.4f}")


X_features dtype: float32
y dtype: float32


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 20ms/step

Sequential_NN general predictions for PART = 0.4:
     MORENA  PRIANPRD    OTROS
0  0.558886  0.414394  0.08102
95% Confidence Intervals for Sequential_NN at PART = 0.4:
Lower - MORENA: 0.5446, PRIANPRD: 0.4024, OTROS: 0.0777
Upper - MORENA: 0.5717, PRIANPRD: 0.4283, OTROS: 0.0841

LSTM general predictions for PART = 0.4:
     MORENA  PRIANPRD    OTROS
0  0.545843  0.414387  0.07113
95% Confidence Intervals for LSTM at PART = 0.4:
Lower - MORENA: 0.5344, PRIANPRD: 0.4015, OTROS: 0.0674
Upper - MORENA: 0.5581, PRIANPRD: 0.4256, OTROS: 0.0751

Sequential_NN general predictions for PART = 0.5:
     MORENA  PRIANPRD     OTROS
0  0.576058   0.41382  0.087863
95% Confidence Intervals for Sequential_NN at PART = 0.5:
Lower - MORENA: 0.5646, PRIANPRD: 0.4021, OTROS: 0.0849
Upper - MORENA: 0.5894, PRIANPRD: 0.4263, OTROS: 0.0918

LSTM general predictions for PART = 0.5:
     MORENA  PRIANPRD 